In [32]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
df=pd.read_csv( "Time_series_analysis_and_forecast_DATASET.csv" ) 

In [33]:
df

,FullDate,Tmax,SysLoad,GasPrice,ElecPrice
0,2010-09-01 00:00:00,15.8,1688.21500,3.6900,23.710
1,2010-09-01 01:00:00,15.8,1669.19500,3.6900,22.635
2,2010-09-01 02:00:00,15.8,1491.98000,3.6900,22.565
3,2010-09-01 03:00:00,15.8,1330.76000,3.6900,18.910
4,2010-09-01 04:00:00,15.8,1247.94000,3.6900,18.030
...,...,...,...,...,...
92011,2021-02-28 19:00:00,24.3,449.63625,5.4848,117.925
92012,2021-02-28 20:00:00,24.3,815.59975,5.4848,87.880
92013,2021-02-28 21:00:00,24.3,745.12325,5.4848,62.060
92014,2021-02-28 22:00:00,24.3,694.95150,5.4848,62.000


In [34]:
# reformat the datetime
df['FullDate']=pd.to_datetime(df['FullDate'])

In [35]:
#remove the outliers
df = df[(df.ElecPrice > df.ElecPrice.quantile(.05)) & (df.ElecPrice < df.ElecPrice.quantile(.95)) ]

In [36]:
# Resample to daily frequency
df = df.resample('D', on='FullDate').sum()

In [37]:
df = df.reset_index()
df.head()

,FullDate,Tmax,SysLoad,GasPrice,ElecPrice
0,2010-09-01,363.4,39557.790,84.87,608.705
1,2010-09-02,369.6,38898.990,88.56,570.810
2,2010-09-03,384.3,38068.815,93.45,493.875
3,2010-09-04,274.0,30940.035,67.80,464.490
4,2010-09-05,298.0,31132.120,89.00,479.035


In [38]:
df1 = df[['FullDate', 'ElecPrice']].copy()
df1.rename(columns={'FullDate': 'ds', 'ElecPrice': 'y'}, inplace=True)

In [39]:
train = df1[:2757]
test = df1[2757:]

In [40]:
from prophet import Prophet

In [41]:
model = Prophet()
model.fit(train[['ds', 'y']])

23:02:43 - cmdstanpy - INFO - Chain [1] start processing
23:02:44 - cmdstanpy - INFO - Chain [1] done processing


In [44]:
# make prediction

future = model.make_future_dataframe(periods=len(test))
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
3829,2021-02-24,3764.072979,2874.754370,4784.007932
3830,2021-02-25,3766.310600,2870.421076,4770.733901
3831,2021-02-26,3773.788775,2882.224745,4726.545279
3832,2021-02-27,3683.978443,2852.382187,4696.771224
3833,2021-02-28,3620.348843,2707.022286,4582.032813


In [46]:
# evaluate model performance

from sklearn.metrics import mean_squared_error, mean_absolute_error
prediction = forecast[2757:]['yhat'].values
test_set = test['y'].values
mse = mean_squared_error(test_set, prediction)
mae = mean_absolute_error(test_set, prediction)
print('Mean Squared Error', mse)
print('Mean Absolute Error', mae)

Mean Squared Error 3226815.859700519
Mean Absolute Error 1544.373896377224
